# Machine Learning Workshop

Here we will walk through an example of a machine learning workflow following five steps:

<img src="../_img/ml_workflow.png" alt="ML Workflow" width="800"/>

For more detailed information on the Shiu Lab's ML pipeline, including explanations of all output files,
check out the [README](https://github.com/ShiuLab/ML-Pipeline).

***

## Step 0. Set up Jupyter notebook & software

Check out this [**guide**](../Tutorial/README.md) to learn how to set up Jupyter notebook and the software needed to run the Shiu Lab's ML pipeline.


***

![Step 1](../_img/step1.png "ML Workflow step 1")

**What do we want to predict?** 

If a gene is annotated as being involved in specialized or general metabolism. 

**What are the labeled instances?**

Tomato genes annotated as being involved in specialized or general metabolism by TomatoCyc.

**What are the predictive features?** 
- duplication information (e.g. number of paralogs, gene family size)
- sequence conservation (e.g. nonsynonymous/synonymouse substitution rates between homologs)
- gene expression (e.g. breadth, stress specific, co-expression)
- protein domain conent (e.g. p450, Aldedh)
- epigenetic modification (e.g. H3K23ac histone marks)
- network properties (# protein-protein interactions, network connectivity).

**What data do we have?**
- 532 tomato genes with specialized metabolism annotation by TomatoCyc
- 2,318 tomato genes with general metabolism annotation by TomatoCyc
- 4,197 features (we are only using a subset of **564** for this workshop)



***

![Step 2](../_img/step2.png "ML Workflow step 2")


In [3]:
## A. Lets look at the data (note, you can do this in excel or R!)
import pandas as pd

d = pd.read_table('data.txt', sep='\t', index_col = 0)

print('Shape of data (rows, cols):')
print(d.shape)

print('\nSnapshot of data:')
print(d.iloc[:6,:5])  # prints first 6 rows and 5 columns

print('\nList of class labels')
print(d['Class'].value_counts())

Shape of data (rows, cols):
(2872, 565)

Snapshot of data:
                Class  Crubella_183_v1.0.csv  FamilySize FamilySize_cat  \
YP_008563134      gen                    0.0    0.010582         medium   
XP_010327628      gen                    0.0    0.000000          small   
XP_010327620  special                    0.0    0.052910         medium   
XP_010327578      gen                    0.0         NaN            NaN   
XP_010327494      gen                    1.0    0.021164         medium   
YP_008563119  special                    0.0    0.000000          small   

              Transferase  
YP_008563134          0.0  
XP_010327628          NaN  
XP_010327620          0.0  
XP_010327578          0.0  
XP_010327494          0.0  
YP_008563119          0.0  

List of class labels
gen        2318
special     532
unknown      22
Name: Class, dtype: int64


**Things to notice:**
- Our data has NAs. ML algorithms cannot handel NAs. We either needs to drop or impute NA values!
- We have binary, continuous, and categorical features in this dataset. A perk of ML models is that they can integrate multiple datatypes in a single model. 
- However, before being used as input, a categorical feature needs to be converted into set binary features using an approach called [one-hot-encoding](https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding). 

*Before One-Hot Encoding:*

| ID   | Class    | Weather   |
|---    |---    |---    |
| instance_A    |  1     | sunny     |
| instance_B    |  0    |  overcast     |
| instance_C   |  0     |  rain    | 
| instance_D   | 1     |  sunny    |

*After One-Hot Encoding:*

| ID   | Class    | Weather_sunny   | Weather_overcast   | Weather_rain   |
|---    |---    |---    |---    |---    |
| instance_A    |  1     | 1     | 0     | 0     |
| instance_B    |  0    |  0     |  1     |  0     |
| instance_C   |  0     |  0    |  0    |  1    | 
| instance_D   | 1     |  1    | 0    | 0    |



***

### Automated data cleaning: ML_preprocess.py

Input
```
-df: your data table
-na_method: how you want to impute NAs (options: drop, mean, median, mode)
-h: show more options
```

In [2]:
# B. Drop/Impute NAs and one-hot-encode categorical features

%run ../ML_preprocess.py -df data.txt -na_method median

Snapshot of input data...
                Class  Crubella_183_v1.0.csv  FamilySize FamilySize_cat  \
YP_008563134      gen                    0.0    0.010582         medium   
XP_010327628      gen                    0.0    0.000000          small   
XP_010327620  special                    0.0    0.052910         medium   
XP_010327578      gen                    0.0         NaN            NaN   
XP_010327494      gen                    1.0    0.021164         medium   

              Transferase  
YP_008563134          0.0  
XP_010327628          NaN  
XP_010327620          0.0  
XP_010327578          0.0  
XP_010327494          0.0  


### Dropping/imputing NAs... ###

Number of columns with NAs: 41
Features dropped because missing > 50.00% of data: ['SQS_PSY']
Number of columns to impute: 40


### One Hot Encoding... ###

Features to one-hot-encode: ['FamilySize_cat']
Dataframe shape (rows, cols) before and after one-hot-encoding:
Before: (2872, 563)
After: (2872, 565)

Number of d

***

## Set aside instances for testing 

We want to set aside a subset of our data to use to test how well our model performed. Note that this is done before feature engineering, parameter selection, or model training. This will ensure our performance metric is entirely independent from our modeling!


### Automated selection of test set: test_set.py

Input
```
-df: your data table
-use: what class labels to include in the test set (we don't want to include unknowns!)
-type: (c) classification or (r) regression
-p: What percent of instances from each class to select for test (0.1 = 10%)
-save: save name for test set
```


In [3]:
# C. Define test set

%run ../test_set.py -df data_mod.txt -use gen,special -type c -p 0.1 -save test_genes2.txt

Holding out 10.0 percent
Pulling test set from classes: ['gen', 'special']
285 instances in test set
finished!


***


![Step 3](../_img/step3.png "ML Workflow step 3")

While one major advantage of ML approaches is that they are robust when the number of features is very large, there are cases where removing unuseful features or selecting only the best features may help you better answer your question. One common issue we see with using feature selection for machine learning is using the whole dataset to select the best features, which results in overfitting! **Be sure you specify your test set so that this data is not used for feature selection!**


### Automated feature selection: Feature_Selection.py

Input
```
-df: your data table
-test: what instances to hold out (i.e. test instances!)
-cl_train: labels to include in training the feature selection algorithm
-type: (c) classification or (r) regression
-alg: what feature selection algorithm to use (e.g. lasso, elastic net, random forest)
-p: Parameter specific to different algorithms (use -h for more information)
-save: save name for list of selected features
```


Here we will use one of the most common feature selection algorithms: LASSO. LASSO requires the user to select the level of sparcity (-p) they want to induce during feature selection, where a larger value will result in more features being selected and a smaller value resulting in fewer features being selected. You can play around with this value to see what it does for your data.  


In [6]:
%run ../Feature_Selection.py -df data_mod.txt -test test_genes.txt -cl_train special,gen -type c -alg lasso -p 0.1 -save top_feat_lasso.txt

Removing testldout instances...
Dropping instances that are not in ['special', 'gen'], changed dimensions from (2587, 566) to (2565, 566) (instance, features).

Snapshot of data:
              Class  Crubella_183_v1.0.csv  FamilySize  Transferase  \
XP_010327628      0                    0.0    0.000000          0.0   
XP_010327620      1                    0.0    0.052910          0.0   
XP_010327578      0                    0.0    0.015873          0.0   
XP_010327494      0                    1.0    0.021164          0.0   
YP_008563119      1                    0.0    0.000000          0.0   
YP_008563115      0                    0.0    0.010582          0.0   

              Exo_endo_phos  
XP_010327628            0.0  
XP_010327620            0.0  
XP_010327578            0.0  
XP_010327494            0.0  
YP_008563119            0.0  
YP_008563115            0.0  
=====* Running L1/LASSO based feature selection *=====
Features selected using LASSO: ['Transferase' 'Exo_endo_ph

***

![Step 4](../_img/step4.png "ML Workflow step 4")

Next we want to determine which ML algorithm (i.e. Support Vector Machine (SVM), Random Forest (RF)) we should use and what parameters needed by those algorithms work best. Importantly, at this stage we **only assess our model performance on the validation data** in order to assure we aren't just selecting the algorithm that works best on our held out testing data. The pipeline will automatically withhold the testing data from the parameter selection (i.e. grid search) step. 

The machine learning algorithms in the ML_Pipeline are implement from [SciKit-Learn](https://scikit-learn.org/stable/), which has excellent resources to learn more about the ins and outs of these algorithms.

Algorithms available in the pipeline are: Support Vector Machine (linear: SVM, polynomial: SVMpoly, radial basis function: SVMrbf), Random Forest (RF), Gradient Tree Boosting (GB), and Logistic Regression (LogReg).


### Parameter Selection

There are two general strategies for parameter selection: the grid search (default option: left) and the random search (use "-gs_type random": right):
![Grid search](../_img/grid_rand_search.png "Grid Search")
*Image: Bergstra & Bengio 2012; used under CC-BY license*


### Training and Validation

Training and validation is done using a [cross-validation (CV)](https://towardsdatascience.com/cross-validation-70289113a072) scheme. CV is useful because it makes good use of our data (i.e. uses all non-test data for training at some point) but also makes sure we are selecting the best parameters/algorithms on models that aren't overfit to the training data. Here is a visual to demonstrate how CV works (with 10-cv folds in this example):

![Cross Validation](../_img/cross_validation.png "Cross validation")


### Automated parameter selection, ML training, validation, & testing:  ML_classification.py/ML_regression.py

Input
```
-df: your data table
-test: what instances to hold out (i.e. test instances)
-cl_train: labels to include in training the feature selection algorithm
-alg: what ML algorithm to use (e.g. SVM, RF, LogReg)
-cv: Number of cross-validation folds (default = 10, use fewer if data set is small)
-save: save name for list of selected features
```

**There are many functions available within the pipeline that are not described in this workshop. For more options run:**

```
python ML_classification.py -h
```


In [6]:
%run ../ML_classification.py -df data_mod.txt -test test_genes.txt -cl_train special,gen -alg SVM -cv 5 -gs true -gs_reps 10 -n 10

Removing test instances to apply model on later...
Snapshot of data being used:
                Class  Crubella_183_v1.0.csv  FamilySize  Transferase  \
XP_010327628      gen                    0.0    0.000000          0.0   
XP_010327620  special                    0.0    0.083333          0.0   
XP_010327578      gen                    0.0    0.025000          0.0   
XP_010327494      gen                    1.0    0.033333          0.0   
YP_008563119  special                    0.0    0.000000          0.0   

              Exo_endo_phos  
XP_010327628            0.0  
XP_010327620            0.0  
XP_010327578            0.0  
XP_010327494            0.0  
YP_008563119            0.0  


CLASSES: ['gen' 'special']
POS: special type:  <class 'str'>
NEG: gen type:  <class 'str'>

Balanced dataset will include 478 instances of each class


===>  Grid search started  <===
Round 1 of 10
Round 2 of 10
Round 3 of 10
Round 4 of 10
Round 5 of 10
Round 6 of 10
Round 7 of 10
Round 8 of 10
Rou

**Results Breakdown**

The output here includes the results on both the validation data (selected randomly interally during the modeling process) and on the held out testing data. Since now we are just selecting the best algorithm/parameters, only look at the validation scores. 

SS doing AUC-ROC. I'll talk about F1 and AUC-ROC, mention many other performance metrics available. 

** Comparing algorithms **

Running the same script (only changing **-alg XXX**), performance on the validation data using other algorithms:

| Alg  	| F1  	| AUC-ROC  	|
|---	|---	|---	|
| RF  	| 0.757  	| 0.824  	|
| LogReg  	| 0.862  	| 0.921  	|
| SVMpoly  	|   	|   	|
| SVM  	| 0.848  	| 0.898  	|


** performed best on the validation data so we will select that algorithm! **

![Step 5](../_img/step5.png "ML Workflow step 5")

Now that we have our best performing algorithm, we will run the pipeline one more time, but with more replicates and we will use it to predict our unknown genes. 

In [ ]:
%run ../ML_classification.py -df data_mod.txt -test test_genes.txt -cl_train special,gen -apply unknown -plots T -alg SVM -cv 5 -gs true -gs_reps 10 -n 50

**Let's check out our results**

(For a detailed description of the content of the pipeline output see the [README](../README.md)

### What if you do things differently

Top few features
Top 50 ish features

Use smaller balenced data set

### Advanced Topics
- multiclass
- transfer learning
- venn diagrams


## 6.D Visualizing Your Results

There are a number of vizualization tools available in the ML-Pipeline. We will describe just some of them here!

**Generate Plots while training and testing your models**

1. One optional parameter for both the ML_classification.py and ML_regression.py pipelines is -plots T/F. The default is False, but if you want to generate some plots directly during your ML run, set -plots T. For classification you will get an [AUC-ROC plot](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) and an [AUC-PRC plot](https://classeval.wordpress.com/introduction/introduction-to-the-precision-recall-plot/). 

2. In the ML_classification.py pipeline, another plotting option is to auto-generate a [confusion matrix heatmap](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62) (-cm), just set -cm T

**Generate plots post-model building**
1. **ML_plots.py** allows you to generate AUC-ROC and AUC-PRC plots from multiple runs, for example if you wanted to compare performance of different algorithms or using different sets of features. For example, we can compare the results from our classification model with all 200 features with the one generated using the top 15 features:

```
ML_plots.py [SAVE_NAME] [POS] [NEG] [M1_name] [PATH_M1_scores] [M2_name] [PATH_M2_scores]... [Mn_name] [PATH_Mn_scores]
```

In [ ]:
%run ../ML_plots.py binary_plots 1 0 all data_binary.txt_SVM_scores.txt top15 data_binary_SVM_top15_scores.txt

2. **compare_classifiers.py** takes in the results from different classification models and generates a list of which instances were correctly and incorrectly predicted by each model and generates a venn diagram showing the overlap.

```
ML_plots.py -scores PATH_M1_scores,PATH_M2_scores,...,PATH_Mn_scores -ids M1_name,M2_name,...,Mn_name -save [SAVE_NAME]
```

In [ ]:
%run ../compare_classifiers.py  -scores data_binary.txt_SVM_scores.txt,data_binary_SVM_top15_scores.txt -ids all,top15 -save binary_comp

These results indicate that our models using all and the top 15 features both predicted 88 instances correctly as positive, but only 82 of those instances were the same for both models, meaning they each uniquely correctly classified 6 instances as positive.

Advanced Topics:

- One parameter that can be adjusted in the ML_classification.py or ML_regression.py script is how many cross validation folds you want to include (-cv). The default and a commonly used fold number is -cv 10, however, if you have a small dataset, using fewer folds (-cv 5) may perform better. In the extreme case, if you have very few instances to train on, you can set -cv equal to the number of instances in your dataset, allowing you to perform leave one out LOO cross validation.